<a href="https://colab.research.google.com/github/jon-nowacki/Optimization-Models/blob/main/Just_In_Time_Inventory_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inventory Cost Minimization & Time Series

Inventory Cost Minimization with Time Series

Given a time-series of monthly forecasted demand, minimize the total cost of inventory, assuming the for each unit

* Holding cost: $2 / month

* Purchase price: $5 / unit

Variables
* 𝑝_𝑖_  : "Number of units of inventory to purchase for month " 𝑖
* 𝑒_𝑖_ : Number of units of inventory held at the end of month 𝑖
Parameters
* 𝑑_𝑖_ : Demand for month 𝑖

Contraints:
* inventory = previous months inventory + purchased - demand



### Install Dependencies
* 14 seconds
* 14 seconds

Total
* 30 seconds

In [ ]:
!pip install pyomo > /dev/null

In [ ]:
!apt-get install -y -qq glpk-utils > /dev/null


### Import Pyomo Code

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

### Minimization Model

Do not modify, this is working code. This is the only code to comment out:

```
model.constraints.add(model.purchase_qty[month]>=5) # none of the months have ZERO ORDER !??!?!?!?
```

In [ ]:
model=ConcreteModel(name='Inventory-MIN')
demand=[4,3,2,1,2,3,4] # Time series prediction
months=len(demand)

holding_cost=1

purchase_cost=2

model.purchase_qty=Var(range(months), domain=NonNegativeIntegers) # Not NonNegativeReals

# ending quantity of inventory per month
model.end_inventory=Var(range(months), domain=NonNegativeIntegers)

model.cost=Objective(expr=sum(
    model.purchase_qty[month]* # Purchase quantity
    purchase_cost+ # Purchase price
    model.end_inventory[month]* # previous inventory
    holding_cost # Holding cost
    for month in range(1,months,1)), sense=minimize) # start at 1, stop at months, iterate by 1

model.constraints=ConstraintList() # empty list of constraints

model.constraints.add(model.end_inventory[0] == 0)

minimum_inventory=3
for month in range(1,months,1):
  next_end_inventory = model.end_inventory[month-1]+model.purchase_qty[month]-demand[month]
  model.constraints.add(model.end_inventory[month] == next_end_inventory)
  # here is where we define a minimum inventory level
  model.constraints.add(model.end_inventory[month]>=minimum_inventory)
  # must buy in bulk
  model.constraints.add(model.purchase_qty[month]>=5) # none of the months have ZERO ORDER !??!?!?!?
# Solve the model
SolverFactory('glpk',executable='/usr/bin/glpsol').solve(model).write()

buffer="# ---------------------------------------------"
print("")
print(buffer)
print("# How much we purchase every month")
model.purchase_qty.display()
print("")
print(buffer)
print("# Inventory at end of month")
model.end_inventory.display()
print(buffer)
print("# Cost")
print(buffer)
model.cost.display()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 125.0
  Upper bound: 125.0
  Number of objectives: 1
  Number of constraints: 19
  Number of variables: 13
  Number of nonzeros: 31
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.017571210861206055
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

### Bulk Purchase Optimizing
This section is in development

In [ ]:
from pyomo.environ import *

model = ConcreteModel(name='Inventory-MIN')
demand = [4, 3, 2, 1, 2, 3, 4]
months = len(demand)

holding_cost = 1
purchase_cost = 2

model.purchase_qty = Var(range(months), domain=NonNegativeIntegers)
model.end_inventory = Var(range(months), domain=NonNegativeIntegers)
model.indicator = Var(range(months), domain=Binary)

model.cost = Objective(expr=sum(
    model.purchase_qty[month] * purchase_cost +
    model.end_inventory[month] * holding_cost
    for month in range(1, months)
), sense=minimize)

model.constraints = ConstraintList()
model.constraints.add(model.end_inventory[0] == 0)

minimum_inventory = 3
min_purchase_quantity = 5

for month in range(1, months):
    next_end_inventory = model.end_inventory[month - 1] + model.purchase_qty[month] - demand[month - 1]
    model.constraints.add(model.end_inventory[month] == next_end_inventory)
    model.constraints.add(model.end_inventory[month] >= minimum_inventory)



    # Ensure purchase quantity is at least 'min_purchase_quantity' if demand exceeds inventory
    M = 1000  # A large constant to handle the indicator constraint
    model.constraints.add(
        model.purchase_qty[month] >= min_purchase_quantity - M * (1 - model.indicator[month])
    )
    model.constraints.add(
        demand[month] - model.end_inventory[month] <= M * model.indicator[month]
    )

# Solve the model
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

buffer="# ---------------------------------------------"
print("")
print(buffer)
print("# How much we purchase every month")
print(buffer)
model.purchase_qty.display()
print("")
print(buffer)
print("# Inventory at end of month")
print(buffer)
model.end_inventory.display()
print(buffer)
print("# Cost")
print(buffer)
model.cost.display()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 57.0
  Upper bound: 57.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 19
  Number of nonzeros: 49
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.005289793014526367
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

### Scratch
I do not care about this code, development only